In [10]:
import tensorflow as tf

import yaml

from sklearn import datasets, linear_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA

import csv

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
import os

from matplotlib.image import imread
import matplotlib.pyplot as plt
%matplotlib inline

import csv
import math

import matplotlib.pyplot as plt
import numpy as np

from keras import applications
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input

import cv2
from skimage import color
from skimage.transform import resize

In [3]:
model = VGG16(weights='imagenet', include_top=False)

import csv
with open("pac_guide.csv") as file:
    reader = csv.reader(file)
    i = 0
    image_paths = []
    camera_nums = []
    labels = []
    
    for row in reader:
        index = row[0]
        camera_num = row[1]
        label = row[2]
        filename = row[3]
        image_paths.append(filename)
        camera_nums.append(camera_num)
        labels.append(label)
        #print(index, camera_num, label, filename)
print(len(image_paths)) # number of rows: 112036
print(len(camera_nums))

sub_dir_hash = {}
for i, sub_dir in enumerate(np.load('sub_dirs.npy')):
    sub_dir_hash[sub_dir] = i

print(sub_dir_hash)


58040320/58889256 [============================>.] - ETA: 0s112036
112036
{'52': 8, '39': 7, '06': 2, '63': 11, '04': 1, '62': 10, '72': 12, '02': 0, '23': 6, '59': 9, '08': 3, '15': 5, '11': 4}


In [21]:
model_num = '15'

In [22]:
train_indices = np.load('pac_arrays/train_' + str(model_num) + '.npy')
test_indices = np.load('pac_arrays/test_' + str(model_num) + '.npy')
print(len(train_indices))
print(len(test_indices))
num_images = len(train_indices) + len(test_indices)

1980
660


In [23]:
def depth_map_to_image(depth_map):
    img = cv2.normalize(depth_map, depth_map, 0, 1, cv2.NORM_MINMAX)
    img = np.array(img * 255, dtype=np.uint8)
    orig_img = img
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.applyColorMap(img, cv2.COLORMAP_OCEAN)
    #img = resize(img, 224, 224)

    return orig_img, img

In [24]:

cnn_feats = np.zeros((num_images, 512 * 7 * 7))

counter = 0

for index in train_indices:

    data = np.load(image_paths[int(index)])
    depth_map = data['x'].astype(np.float32)
    orig_img, ocean = depth_map_to_image(depth_map)
    ocean = resize(ocean, (224, 224), mode='constant')
    x = image.img_to_array(ocean)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)

    # Reshape features into 2D tensor
    reshaped_features = features.reshape(1, 512*7*7)
    cnn_feats[counter, :] = reshaped_features

    counter+=1
    
for index in test_indices:

    data = np.load(image_paths[int(index)])
    depth_map = data['x'].astype(np.float32)
    orig_img, ocean = depth_map_to_image(depth_map)
    ocean = resize(ocean, (224, 224), mode='constant')
    x = image.img_to_array(ocean)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)

    # Reshape features into 2D tensor
    reshaped_features = features.reshape(1, 512*7*7)
    cnn_feats[counter, :] = reshaped_features

    counter+=1

print(cnn_feats.shape)

# reduce dimensionality using principal components analysis (PCA)

pca = PCA(n_components=256)
pca.fit(cnn_feats)
cnn_feats_compressed = pca.transform(cnn_feats)

np.save('PAC_cnn_pca_components_' + str(model_num), pca.components_)
np.save('PAC_cnn_pca_features_' + str(model_num), cnn_feats_compressed)

(2640, 25088)
